In [41]:
# step 1
from pyscf import gto, scf

mol = gto.Mole()

mol.atom = """
                C    4.80585    -0.85311    1.96913  
                H    5.1625    -1.86192    1.96913  
                H    5.16252    -0.34871    2.84278  
                H    5.16252    -0.34871    1.09548  
                C    3.26585    -0.85309    1.96913  
                H    2.9092    0.15572    1.97108  
                H    2.90918    -1.35579    1.0945  
                C    2.75251    -1.58148    3.22512  
                H    3.10949    -1.079    4.09975  
                H    3.10885    -2.5904    3.22298  
                C    1.21251    -1.58099    3.22539  
                H    0.85553    -2.0834    2.35072  
                H    0.85616    -0.57208    3.22762  
                H    0.71486    -2.13574    4.355959
    """

mol.basis = "6-31g"
mol.build()

mf = scf.RHF(mol).run()

converged SCF energy = -157.201013620905


In [42]:
# step 2
from pyscf import mp
import timeit
start = timeit.default_timer()
pt = mp.MP2(mf).run()
stop = timeit.default_timer()
print("Time: ", stop - start)

E(MP2) = -157.577751503194  E_corr = -0.376737882288067
Time:  0.19799925700044696


In [43]:
# step 3
import numpy as np
from pyscf import ao2mo

import timeit
start = timeit.default_timer()
ntot = mf.get_occ().shape[0]
nocc = np.count_nonzero(mf.get_occ())
nvir = ntot - nocc
e = mf.mo_energy

# step 4
aoints = mol.intor("int2e")
start = timeit.default_timer()
moints = ao2mo.incore.full(aoints, mf.mo_coeff)

# step 5
ecorr = 0.0
for i in range(nocc):
    for j in range(i, nocc):
        for a in range(nocc, ntot):
            for b in range(nocc, ntot):
                c = (2 - int(i == j))
                ecorr += c * (moints[i, a, j, b] \
                           * (2 * moints[i, a, j, b] - moints[i, b, j, a])) \
                           / (e[i] + e[j] - e[a] - e[b])

                
stop = timeit.default_timer()
print(ecorr)
print("Time: ", stop - start)

-0.37673788228780597
Time:  2.647096175000115


In [44]:
print(nocc)
print(nvir)

17
39


In [45]:
start = timeit.default_timer()

# step 3
ntot = mf.get_occ().shape[0]
nocc = np.count_nonzero(mf.get_occ())
nvir = ntot - nocc
e = mf.mo_energy
co = np.asarray(mf.mo_coeff[:,:nocc])
cv = np.asarray(mf.mo_coeff[:,nocc:])

# step 4
moints = ao2mo.general(mf._eri, (co, cv, co, cv))
moints = moints.reshape(nocc, nvir, nocc, nvir)

moints2 = moints.transpose(0,3,2,1)
et  = np.asarray(e[:nocc, None,  None,  None] \
                 - e[None, nocc:, None,  None] \
                 + e[None, None, :nocc,  None] \
                 - e[None, None,  None,  nocc:])

# step 5
mat = moints * (2 * moints - moints2) / et

print(np.sum(mat))
stop = timeit.default_timer()
print("Time: ", stop - start)

-0.3767378822880667
Time:  0.19784339599937084


In [46]:
np.prod(moints.shape)

439569

In [3]:
len(mf.mo_energy)

56

In [7]:
(55 * 56 / 2) 

2371600.0

In [10]:
56 / 8

7.0

In [24]:
import numpy as np

a = np.array([1, 2])
print(a)

[1 2]


In [31]:
print(a[:, None, None] - a[None, :, None] + a[None, None, :])

[[[1 2]
  [0 1]]

 [[2 3]
  [1 2]]]


In [ ]:
np.einsum